# Introduçãos às CNNs no PyTorch

## Objetivos

Este notebook contém exemplo numérico de uma rede com uma camada convolucional e uma camada densa. A camada convolucional possui ativação reLU e max-pooling. Já a camada densa possui uma única saída com ativação sigmóide.

Com esse exemplo, aprende-se a:
- construir a rede utilizando subclasses de Module do PyTorch
- inicializar os pesos e biases da rede convolucional e da rede densa
- visualizar os dados intermediários da rede

## Importação dos Módulos

In [1]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.functional as F

from torch.autograd import Variable
from collections import OrderedDict
from IPython.core.display import display, HTML

np.set_printoptions(precision=3) # ponto flutuante com 3 casas para facilitar a impressão

In [2]:
# verifica se a GPU está disponível
use_gpu = torch.cuda.is_available()
print("Usando GPU:", use_gpu)

Usando GPU: True


## Definição da Rede, camadas convolucionais e densas

### Definição de camadas

Em redes neurais, uma camada é usualmente um neurônio, que inclui a soma de multiplicação de
pesos ou convolução e uma ativação usualmente não linear. Podemos dizer que o max-pooling 
também faz parte da camada.

A rede que iremos utilizar neste exemplo possui 2 camadas: uma convolucional e outra densa.
A camada convolucional terá ativação reLU e um max-pooling, já a camada densa terá uma
ativação sigmóide.

### Rede a ser implementada

<img src='../figures/RedeIntroKeras.png', width=600pt>

### Criando redes np PyTorch

Uma CNN no PyTorch é criada utilizando uma classe que é subclasse do *torch.nn.Module*. Essa classe poderá ter variáveis que também são instâncias de subclasses de *torch.nn.Module*.

- *torch.nn.Conv2d*
- *torch.nn.ReLU*
- *torch.nn.MaxPool2d*
- *torch.nn.Linear*
- *torch.nn.Sequential*

Todas estas classes são também subclasses de *torch.nn.Module* e podem ser instanciadas no contrutor para serem utilizadas na classe (ver [Documentação torch.nn](http://pytorch.org/docs/master/nn.html))

### Implementação do modelo

In [3]:
class Model(nn.Module):
    
    # Definição de Modules que serão utilizados na rede
    def __init__(self):
        super(Model, self).__init__()
        
        # camada convolucional
        self.conv = nn.Conv2d(1, 3, (2, 2), padding=0, bias=True)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d((2, 2))
        
        # camada densa
        self.dense = nn.Linear(3*4, 1, bias=True)
        self.sigmoid = nn.Sigmoid()
    
    # Método de definição obrigatória
    # Sequencia que será efetivamente executada para obter a saída da rede
    def forward(self, x):
        # passa os dados pela camada convolucional
        x = self.conv(x)
        x = self.relu(x)
        x = self.max_pool(x)
        
        # faz o flatten dos dados
        x = x.view(-1, 3*4)
        
        # passa os dados pela camada densa
        x = self.dense(x)
        x = self.sigmoid(x)
        
        return x
    
    # Método opcional utilizado para mostrar a saída de cada parte da rede
    def modules_output(self, x):
        output = OrderedDict()        

        # passa os dados pela camada convolucional
        x = self.conv(x)
        output['conv'] = x
        
        x = self.relu(x)
        output['relu'] = x
        
        x = self.max_pool(x)
        output['max_pool'] = x
        
        # faz o flatten dos dados
        x = x.view(-1, 3*4)
        output['flatten'] = x
        
        # passa os dados pela camada densa
        x = self.dense(x)
        output['dense'] = x
        x = self.sigmoid(x)
        output['sigmoid'] = x

        return output

In [4]:
model = Model()
print(model)

Model (
  (conv): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
  (relu): ReLU ()
  (max_pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (dense): Linear (12 -> 1)
  (sigmoid): Sigmoid ()
)


### Outro modelo encapsulando as camadas em uma *Sequential*

In [5]:
class Model2(nn.Module):

    # Definição de Modules que serão utilizados na rede
    def __init__(self):
        super(Model2, self).__init__()
        
        # camada convolucional
        self.layer1 = nn.Sequential(OrderedDict([
            ('conv', nn.Conv2d(1, 3, (2, 2), padding=0, bias=True)),
            ('relu', nn.ReLU()),
            ('max_pool', nn.MaxPool2d((2, 2)))
        ]))
        
        # camada densa
        self.layer2 = nn.Sequential(OrderedDict([
            ('dense', nn.Linear(3*4, 1, bias=True)),
            ('sigmoid', nn.Sigmoid())
        ]))

    # Método de definição obrigatória
    # Sequencia que será efetivamente executada para obter a saída da rede
    def forward(self, x):
        # passa os dados pela camada convolucional
        x = self.layer1(x)
        # faz o flatten dos dados
        x = x.view(-1, 3*4)
        # passa os dados pela camada densa
        x = self.layer2(x) 
        
        return x

In [6]:
model2 = Model2()
print(model2)

Model2 (
  (layer1): Sequential (
    (conv): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
    (relu): ReLU ()
    (max_pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layer2): Sequential (
    (dense): Linear (12 -> 1)
    (sigmoid): Sigmoid ()
  )
)


# Configurando a rede neural com convolução e densa

## Definição dos parâmetros da rede

### Imagem de Entrada: 5 linhas e 6 colunas

In [7]:
X = np.array([[[[0,0,0,0,0,0],
                [0,0,1,0,0,0],
                [0,0,0,0,0,0],
                [0,0,0,0,-1,0],
                [0,0,0,0,0,0]]]])

(n_samples, n_channels, img_height, img_width) = X.shape

print('X.shape=', X.shape)
print('X:\n', X)

X.shape= (1, 1, 5, 6)
X:
 [[[[ 0  0  0  0  0  0]
   [ 0  0  1  0  0  0]
   [ 0  0  0  0  0  0]
   [ 0  0  0  0 -1  0]
   [ 0  0  0  0  0  0]]]]


### Kernel da convolução

In [8]:
# número de filtros
n_filters = 3    

# comprimento e largura dos filtros
k_height = k_width = 2 
kernel_shape = (k_height, k_width)

W_conv = np.array([[[1,2],
                 [3,4]],
                [[5,6],
                 [7,8]],
                [[9,10],
                 [11,12]]]).reshape(n_filters,1,k_height,k_width)

print('W_conv.shape (n_filters, n_channels, k_height, k_width):', W_conv.shape)
print('W_conv:\n', W_conv)

W_conv.shape (n_filters, n_channels, k_height, k_width): (3, 1, 2, 2)
W_conv:
 [[[[ 1  2]
   [ 3  4]]]


 [[[ 5  6]
   [ 7  8]]]


 [[[ 9 10]
   [11 12]]]]


### Bias da Convolução

In [9]:
# valor de bias
f_bias = 0.1     
bias_conv = np.arange(1,n_filters+1) * f_bias
print("Bias da convolução:",bias_conv)

Bias da convolução: [ 0.1  0.2  0.3]


### Pesos para a camada densa

In [10]:
# após o max pooling, são 3 imagens 2x2 = 12
W_dense = np.arange(12).reshape(1, 12)
print("Pesos da camada densa:\n",W_dense)

Pesos da camada densa:
 [[ 0  1  2  3  4  5  6  7  8  9 10 11]]


### Bias para a camada densa

In [11]:
bias_dense = np.ones(1) * f_bias
print("Bias da camanda densa:",bias_dense)

Bias da camanda densa: [ 0.1]


## Imprimindo os valores dos dados das camadas internas da rede

### Instancia a rede e aplica os pesos definidos

In [12]:
# Mostra o shape dos pesos das camadas
display(HTML('<h3>Shape dos pesos</h3>'))
print('{:11} {}'.format('W_conv:', W_conv.shape))
print('{:11} {}'.format('bias_conv:', bias_conv.shape))
print('{:11} {}'.format('W_dense:', W_dense.shape))
print('{:11} {}'.format('bias_dense:', bias_dense.shape))
print()

# cria o modelo
model = Model()

# Dictionary com os pesos para a rede
my_weights = OrderedDict([
    ('conv.weight',  torch.FloatTensor(W_conv.astype(float))),
    ('conv.bias',    torch.FloatTensor(bias_conv.astype(float))),
    ('dense.weight', torch.FloatTensor( W_dense.astype(float))),
    ('dense.bias',   torch.FloatTensor(bias_dense.astype(float))),
])

# aplica os pesos criados à rede
model.load_state_dict(my_weights)

#Mostra pesos aplicado ao modelo
display(HTML('<h3>Pesos aplicados à rede</h3>'))
for param_name, param in model.state_dict().items():
    display(HTML('<h4>{}</h4>'.format(param_name)))
    np_param = param.numpy()
    print(np_param)
    print('\nshape:', np_param.shape)  

#print(model.state_dict())

W_conv:     (3, 1, 2, 2)
bias_conv:  (3,)
W_dense:    (1, 12)
bias_dense: (1,)



[[[[  1.   2.]
   [  3.   4.]]]


 [[[  5.   6.]
   [  7.   8.]]]


 [[[  9.  10.]
   [ 11.  12.]]]]

shape: (3, 1, 2, 2)


[ 0.1  0.2  0.3]

shape: (3,)


[[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.]]

shape: (1, 12)


[ 0.1]

shape: (1,)


### Mostra as saídas de cada parte da execução da rede

In [13]:
display(HTML('<h3>Saídas de cada camada da rede</h3>'))

# Transforma X em Variable
X = Variable(torch.FloatTensor(X.astype(float)))

# Resultados para cada camada
for module_name, output in model.modules_output(X).items():
    display(HTML('<h4>{}</h4>'.format(module_name)))
    np_output = output.data.numpy()
    print(np_output)
    print('\nshape:', np_output.shape)    

[[[[  0.1   4.1   3.1   0.1   0.1]
   [  0.1   2.1   1.1   0.1   0.1]
   [  0.1   0.1   0.1  -3.9  -2.9]
   [  0.1   0.1   0.1  -1.9  -0.9]]

  [[  0.2   8.2   7.2   0.2   0.2]
   [  0.2   6.2   5.2   0.2   0.2]
   [  0.2   0.2   0.2  -7.8  -6.8]
   [  0.2   0.2   0.2  -5.8  -4.8]]

  [[  0.3  12.3  11.3   0.3   0.3]
   [  0.3  10.3   9.3   0.3   0.3]
   [  0.3   0.3   0.3 -11.7 -10.7]
   [  0.3   0.3   0.3  -9.7  -8.7]]]]

shape: (1, 3, 4, 5)


[[[[  0.1   4.1   3.1   0.1   0.1]
   [  0.1   2.1   1.1   0.1   0.1]
   [  0.1   0.1   0.1   0.    0. ]
   [  0.1   0.1   0.1   0.    0. ]]

  [[  0.2   8.2   7.2   0.2   0.2]
   [  0.2   6.2   5.2   0.2   0.2]
   [  0.2   0.2   0.2   0.    0. ]
   [  0.2   0.2   0.2   0.    0. ]]

  [[  0.3  12.3  11.3   0.3   0.3]
   [  0.3  10.3   9.3   0.3   0.3]
   [  0.3   0.3   0.3   0.    0. ]
   [  0.3   0.3   0.3   0.    0. ]]]]

shape: (1, 3, 4, 5)


[[[[  4.1   3.1]
   [  0.1   0.1]]

  [[  8.2   7.2]
   [  0.2   0.2]]

  [[ 12.3  11.3]
   [  0.3   0.3]]]]

shape: (1, 3, 2, 2)


[[  4.1   3.1   0.1   0.1   8.2   7.2   0.2   0.2  12.3  11.3   0.3   0.3]]

shape: (1, 12)


[[ 281.5]]

shape: (1, 1)


[[ 1.]]

shape: (1, 1)


# Sugestões de atividades

1. Quais as vantagens de utilizar Sequential na definição do modelo?
2. Como calcular o número de parâmetros da rede?
3. Retirar o bias e recalcular o número de parâmetros a serem treinados
4. A rede foi projetada para aceitar entrada com shape (1,5,6). Mudar a rede para aceitar:
   a) entrada com shape (1,6,6)
   b) entrada com shape (3,6,6)
5. Inserir mais uma camada convolucional com 4 filtros de saída e kernel (3,3)

# Referências

- [PyTorch Documentation](http://pytorch.org/docs/master/index.html)


# Aprendizados com este notebook
